In [1]:
from google.colab import files
uploaded = files.upload()

Saving cses4_cut.csv to cses4_cut (5).csv


In [2]:
import io
import pandas as pd
import numpy as np

data = pd.read_csv(io.BytesIO(uploaded['cses4_cut.csv']))

In [3]:
voted = data['voted'] # Seperating the result and the parameters
del data['voted']
del data['Unnamed: 0']

In [4]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

test = SelectKBest(score_func=chi2, k='all')                # Checking the meaningful parameters that have huge affect on the result
fit = test.fit(data, voted)
feat_Score = fit.scores_

d = {'Feature': data.columns, 'Feature_Score': feat_Score}  # Storing the scores of the features and sorting them to see their importance easily.
df = pd.DataFrame(d)
df.sort_values('Feature_Score', ascending=False)[:15]       # Here we see first 12 features make more sense

,Feature,Feature_Score
24,D2026,137612.525035
25,D2027,48651.854804
27,D2029,41862.441438
19,D2021,23616.010551
9,D2011,18013.967469
28,D2030,11274.361206
20,D2022,9679.996949
26,D2028,8300.820953
21,D2023,7728.432886
13,D2015,6210.004842


In [5]:
reconstructedData = data[['D2026','D2027','D2029','D2021','D2011','D2030','D2022','D2028','D2023','D2015','age','D2016']] # Reconstructing the parametes considering their importance
reconstructedData = reconstructedData.sort_index(axis=1)

In [6]:
from sklearn.model_selection import train_test_split # Spliting the data to train and test the model
Xtrain, Xtest, ytrain, ytest = train_test_split(reconstructedData, voted, test_size=0.2, random_state=1)


In [7]:
from sklearn.linear_model import LogisticRegression

log_R = LogisticRegression()
log_R.fit(Xtrain, ytrain)
log_R_y_head = log_R.predict(Xtest) # Predicting the Y results with the trained model using the test data.

log_Accuracy = accuracy_score(ytest, log_R_y_head) # Compairing the performance of the model by using the predicted values and actual values.
cv_log = cross_val_score(log_R, Xtrain, ytrain, cv=8) # Besides to test and train split, cross validating the model with the train data
cv_log_mean = cv_log.mean() # mean accuracy of the folded model
cv_log_std = cv_log.std() # std of the folded model

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [8]:
from sklearn.naive_bayes import GaussianNB

gaus_NB= GaussianNB()
gaus_NB.fit(Xtrain, ytrain)
gaus_NB_y_head=gaus_NB.predict(Xtest)

gaus_NB_Accuracy = accuracy_score(ytest, gaus_NB_y_head)
cv_gausNB = cross_val_score(gaus_NB, Xtrain, ytrain, cv=8)
cv_gausNB_mean = cv_gausNB.mean()
cv_gausNB_std = cv_gausNB.std()

In [9]:
from sklearn.ensemble import RandomForestClassifier

randFC = RandomForestClassifier()
randFC.fit(Xtrain, ytrain)
radnFC_y_head=randFC.predict(Xtest)

randFC_Accuracy=accuracy_score(ytest,radnFC_y_head)
cv_randFC = cross_val_score(randFC, Xtrain, ytrain, cv=8)
cv_randFC_mean = cv_randFC.mean()
cv_randFC_std = cv_randFC.std()

In [10]:
from sklearn.neighbors import KNeighborsClassifier

n=np.arange(1,26)
Kth_N_Accuracy=np.zeros(len(n))   # Since number of the neighbors has impact on the performance first deciding the optimum number of neighbors.

for i in range(len(n)):
    KNN_C = KNeighborsClassifier(i+1)
    KNN_C.fit(Xtrain, ytrain)
    KNN_C_y_head = KNN_C.predict(Xtest)

    cv_KNN = cross_val_score(KNN_C, Xtrain, ytrain, cv=8)
    cv_KNN_mean = cv_KNN.mean()
    Kth_N_Accuracy[i] = cv_KNN_mean
    



In [11]:
opt_N=Kth_N_Accuracy.argmax()+1 # Choosing the number of neighbors considering the accuracies.


KNN_C = KNeighborsClassifier(opt_N)
KNN_C.fit(Xtrain, ytrain)
KNN_C_y_head = KNN_C.predict(Xtest)

KNN_C_Accuracy = accuracy_score(ytest, KNN_C_y_head)
cv_KNN = cross_val_score(KNN_C, Xtrain, ytrain, cv=8)
cv_KNN_mean = cv_KNN.mean()
cv_KNN_std = cv_KNN.std()

In [12]:
from sklearn.svm import LinearSVC

L_SVC = LinearSVC()
L_SVC.fit(Xtrain, ytrain)
L_SVC_y_head = L_SVC.predict(Xtest)

L_SVC_Accuracy = accuracy_score(ytest, L_SVC_y_head)
cv_L_SVC = cross_val_score(L_SVC, Xtrain, ytrain, cv=8)
cv_L_SVC_mean = cv_L_SVC.mean()
cv_L_SVC_std = cv_L_SVC.std()

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

In [13]:
method_scores = {'Method':['Logistic Regression', 'Gaussian Naive Bayes', 'Random Forest', 'KNN', 'SVM'],
               'y_head Accuracy':[log_Accuracy, gaus_NB_Accuracy, randFC_Accuracy, KNN_C_Accuracy, L_SVC_Accuracy],
               'cv_mean':[cv_log_mean, cv_gausNB_mean, cv_randFC_mean, cv_KNN_mean, cv_L_SVC_mean],
               'cv_std':[cv_log_std, cv_gausNB_std, cv_randFC_std, cv_KNN_std, cv_L_SVC_std]
                         }
valuation = pd.DataFrame(method_scores)
valuation = valuation.sort_values(by='cv_mean', ascending=False)
valuation

,Method,y_head Accuracy,cv_mean,cv_std
2,Random Forest,0.849458,0.856024,0.007478
3,KNN,0.845042,0.850100,0.004153
0,Logistic Regression,0.823364,0.829618,0.005854
1,Gaussian Naive Bayes,0.740666,0.732932,0.013007
4,SVM,0.724609,0.728313,0.201779
